# Notebook 6 - Putting it all together

Although in this notebook we will not go over new topics, we are going to see how we can fully handle a text sentiment classification task. We are going to use many different techniques covered in previous notebooks, that (hopefully) let us develop quite an efficient model. Additionally, we are going to create some visualizations of dataset statistics and model performance, so we can understand the problem better.

We are going to use the dataset containing tweets about US airlines from [kaggle](). The dataset is stored in the CSV file `airline_tweets.csv` and contains tweet text labeled with either *"positive"*, *"neutral"* or *"negative"* sentiment. Besides, it contains (potentially) helpful information like the name of the airline and the user’s location. Let's explore it!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from langdetect import detect
import seaborn as sns
import spacy
import en_core_web_sm

# Data preprocessing & feature engineering

## Dataset overview

In [ ]:
df_tweets = pd.read_csv("./datasets/airline_tweets.csv")

In [ ]:
df_tweets.head()

Ok, so there are many columns with additional information. However, some of their fields are empty. To determine whether we want to include a certain column in the further analysis, we may want to inspect how many fields in these columns are null. Let's do it using the `.info()` method.

In [ ]:
df_tweets.info()

Ok, now we have an overview of the dataset and we can see that columns like `airline_sentiment_gold` or `tweet_coord` are generally empty (contain null values). How about columns like `tweet_location` or `user_timezone`? We can inspect them visually and see if "null" values are stacked or randomly scattered throughout the whole dataset.

In [ ]:
sns.heatmap(df_tweets.isnull(), cbar=False)

In our study, besides "text" and "airline_sentiment" we will use the "airline" and "tweet_location" columns.

In [ ]:
df = df_tweets[[ "airline_sentiment", "airline", "text", "tweet_location"]]

In [ ]:
df.head()

---

## Baseline classification on raw data

Now, let's train the Naive-Bayes classifier without any preprocessing or parameter tuning, and let's see what accuracy score we get.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_tweets.text, df_tweets.airline_sentiment, test_size=0.2, 
                                                                                random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1,1),binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train_counts, y_train)

In [ ]:
y_predicted_counts = mnb.predict(X_test_counts)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    precision = precision_score(y_test, y_predicted, pos_label=None, average='weighted')             
    recall = recall_score(y_test, y_predicted, pos_label=None, average='weighted')
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1


accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

It seems like even without much work we can get not bad results! Let's verify them using cross-validation.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(mnb, X_train_counts, y_train, cv=5)

In [ ]:
scores.mean()

Ok, so the baseline classifier ran on the totally unprocessed dataset gives us **around 75% accuracy** using cross-validation. We will come back to this number later!

From now we aim to get a higher accuracy score. We are going to preprocess data, extract some features and work on the model parameters selection, so all of this work will hopefully let us get better results.

---

## Further dataset exploration

Let's see what is the structure of the dataset in terms of the airline and sentiment.

In [ ]:
sns.catplot(x="airline",kind="count", data=df)

In terms of airlines, the situation doesn't look very bad except for the low number of tweets regarding Virgin America.

In [ ]:
sns.catplot(x="airline_sentiment", kind="count", data=df)

However, when it comes to sentiment, we can see that the dataset is highly imbalanced. We have much more negative tweets than neutral or positive ones. Often the result of such imbalance is the lower accuracy. To partially solve this issue we may want to *resample* the dataset later.

We may also want to see what is the sentiment statistic for each airline.

In [ ]:
sns.factorplot("airline", data=df, hue="airline_sentiment", kind="count")

What we can read from this plot? Actually quite useful information! We can see that the airline people tweet about has an influence on the tweet sentiment. For example, the great majority of all tweets about United Airlines are negative. On the other hand, neutral and positive tweets regarding Delta Airlines outnumber negative samples. Hence, we will want to treat the tweet-related airline as a feature.

---

## Feature extraction - airline 

So how can we use the name of the airline as a feature? One way is to add it at the end of the text. By doing so, we will give a classifier a "hint", that is related to the sentiment. Interestingly, since every tweet mentions one of 6 airlines, they already contain airlines names in the beginning! Let's make sure that it is true for all tweets.

In [ ]:
df_starting_word = df["text"].apply(word_tokenize)
pd.Series(x[1].lower() for x in df_starting_word).value_counts().head(10)

As you can see, the great majority of all tweets start with the airline name. Some tweets do not, but we will ignore them since these are rare cases. In the preprocessing, we will remove "@" characters and make all words lowercase so we will treat "united" and "United" as the same token.

Hence, we can drop the "airline" column, since we will not use it anymore.

In [ ]:
del df["airline"]
df

---

## Feature extraction - location

We can also extract the location from the "tweet_location" column. One may think that since the column already exists, the only thing we need to do is to match it with the label column. In this case, this won't work, since the "tweet_location" column contains geographical names in different formats, sometimes contains even more than one location. We are going to extract them from the column using the `SpaCy` English language model - "en_core_web_sm" (already downloaded in the Notebook 3).

We have already seen named-entity recognition in Notebook 2, so let's apply it here. We will extract all geographical entities (GPE). Let's see the example below.

In [ ]:
text="San Mateo, CA & Las Vegas, NV"
NLP = en_core_web_sm.load()
output = NLP(text)
for item in output.ents:
    print(item.label_, item)

Now, let's create a function, that given a text from the "tweet_location" column returns all found GPE names. If the column field is empty, the function will return "nolocationplaceholder" to consistently specify the lack of the location.

In [ ]:
def filter_location(text):
    if text == "":
        return "nolocationplaceholder"
    else:
        try:
            output = NLP(text)
            locations = []
            for item in output.ents:
                if item.label_ == "GPE":
                    locations.append(str(item))
            if not len(locations):
                return "nolocationplaceholder"
            return locations
        except:
            return "nolocationplaceholder"

Let's create a special series in the dataframe for the extracted locations list (or nolocationplaceholder).

In [ ]:
df["location_ner"] = df.tweet_location.swifter.apply(filter_location)

In [ ]:
df

However, we would like to have each extracted location in a specific column.

In [ ]:
location_df = df["location_ner"].apply(pd.Series)
location_df

Now some concatenation and renaming...

In [ ]:
df = pd.concat([location_df, df], axis=1)

In [ ]:
df.rename(columns={0: 'loc_1', 1: 'loc_2', 2: "loc_3", 3: "loc_4"}, inplace="True")

And done!

In [ ]:
df

So what can we do with these extracted location names? We can try to find a relation between the location name and the sentiment of the tweet. Firstly, let's see the top 20 locations.

In [ ]:
dict(df.loc_1.value_counts().iloc[:20])

Now, let's see if there is any relation between tweets sentiment and the location. We can 

In [ ]:
sns.catplot(x="airline_sentiment",kind="count", data=df[df.loc_1 == "London"])

In [ ]:
sns.catplot(x="airline_sentiment",kind="count", data=df[df.loc_1 == "San Francisco"])

In [ ]:
sns.catplot(x="airline_sentiment",kind="count", data=df[df.loc_1 == "New York"])

It seems like there is no big difference from where the tweet was tweeted since the majority seems to be negative. However, let's take loc_1 as a feature and concatenate it with the tweet text.

In [ ]:
df["text_loc"] = df["text"].astype(str) + " " + df["loc_1"].astype(str)

Also, now, we can drop all other unused columns and just keep our edited text and the airline_sentiment label.

In [ ]:
df = df[["text_loc", "airline_sentiment" ]]

In [ ]:
df

---

## Preprocessing and extracting features from text
Now since we have already extracted the precise location of the tweets (if it was an actual geographical location) we can start with the preprocessing of the raw text.

### Language

Firstly, we will start with determining the actual language of each tweet. The fact that 10 tweets from the top and the bottom are in English doesn't mean that all of them are! We don't want to mix languages since the same words often repeat in different languages and may have different meanings. To detect the language of each tweet we will use the `langdetect` package.

In [ ]:
df['language'] = df["text_loc"].swifter.apply(detect)

In [ ]:
df.language.value_counts()

In [ ]:
df[df.language =="it"]

In [ ]:
df[df.language =="fr"]

It seems like the langdetect was mistaken and the majority of all tweets is actually in English, so **let's not delete anything**.

### Emojis

One interesting text-based feature that can be extracted is the occurrence of emojis with a negative/positive meaning. The intuition behind this is to develop a list of negative emojis and if a tweet contains one of them, mark that tweet with a certain token, that the classifier will (hopefully) relate with the negative sentiment. However, before treating it as a feature we should check if it is related to the sentiment.

To start, we need a way of extracting emojis from the text. Let's explore the example below. 

In [ ]:
import emoji
from emoji import UNICODE_EMOJI
import re

example_emoji_sentence = "@VirginAmerica 👍 Need to start flying to @KCIAirport .  😊😀😃😄"

def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

In [ ]:
extract_emojis(example_emoji_sentence)

We may want to explore what kind of emojis and how joined together occur in our dataset.

In [ ]:
emojis_ = []

In [ ]:
corpus = df.text_loc.tolist()
for i in corpus:
    h = extract_emojis(i)
    if len(h) != 0:
        emojis_.append(h)

In [ ]:
emojis_

Let's create a list of emojis with negative meanings and develop a function that will return the "negemoji" token if there is a negative emoji in the tweet text.

In [ ]:
neg_emojis = "😭,🆘, 😡, 😩, 😞, 😢, 👿, 👎, 😔, 😪, 😫, 😤, 😖, 😠, 💩, 😑, 😕, 😒"

def contains_neg_emoji(text):
    emojis =  ''.join(c for c in text if c in emoji.UNICODE_EMOJI['en'])
    contains_neg = ""
    for e in emojis:
        if e in neg_emojis:
            contains_neg = "negemoji"
            
    return contains_neg

In [ ]:
df["neg_emoji"] = df.text_loc.apply(contains_neg_emoji)

Ok, now let's see how many tweets actually have negative emojis.

In [ ]:
df.neg_emoji.value_counts()

Oops, only 1% of all tweets contain negative emojis!  Let's check the relation between negative emojis and the sentiment of the tweet.

In [ ]:
sns.catplot("neg_emoji", data=df[df["neg_emoji"] == 'negemoji'], hue="airline_sentiment", kind="count")

However, the relation between containing negative emoji and the sentiment of a tweet looks very good - there is a very small number of positive-labeled tweets containing a negative emoji. 

In [ ]:
df

The occurrence of negative emojis probably won't help much the classifier since it occurs in only 1% of all tweets but let's still implement it as a feature!

In [ ]:
df["text_loc"] = df["text_loc"].astype(str) + " " + df["neg_emoji"].astype(str)

In [ ]:
df = df[["text_loc", "airline_sentiment"]]

In [ ]:
df

## Further feature extraction - urls, numbers, hashtags...

Here, we are going to extract more features from the text. If we research that a specific feature discriminates well (is related to the positive or negative sentiment), we will mark a tweet with a specific token - as previously.

### URLs

In [ ]:
import re
def contains_link(text):
    text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr',
                     text)
    text = word_tokenize(text)
    if "httpaddr" in text:
        return "YES"
    else:
        return "NO"

In [ ]:
df["contains_link"] = df.text_loc.apply(contains_link)

In [ ]:
sns.factorplot("contains_link", data=df, hue="airline_sentiment", kind="count")

### Numbers

In [ ]:
def contains_number(text):
        
    text = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'numbr',
                     text)
    
    text = re.sub(r'\d+(\.\d+)?', 'numbr',
                     text)
    text = word_tokenize(text)
    if "numbr" in text:
        return "YES"
    else:
        return "NO"

In [ ]:
df["contains_number"] = df.text_loc.apply(contains_number)

In [ ]:
sns.factorplot("contains_number", data=df, hue="airline_sentiment", kind="count")

### Hashtags

In [ ]:
def contains_hashtag(text):
        
    text = re.sub(r'\b#w+\b', 'hashtg',
                     text)
    
    text = re.sub(r'\d+(\.\d+)?', 'hashtg',
                     text)
    text = word_tokenize(text)
    if "hashtg" in text:
        return "YES"
    else:
        return "NO"

In [ ]:
df["contains_hashtag"] = df.text_loc.apply(contains_hashtag)

In [ ]:
sns.factorplot("contains_hashtag", data=df, hue="airline_sentiment", kind="count")

Sadly, the occurrence of URLs, numbers, or hashtags doesn't seem to have a strong influence on the tweet sentiment. We will not add them as features.

---

## Preprocessing - cleaning

### Standard preprocessing 

Now we need to clean the data. Here, we will replace all URLs, numbers with a standard token. Besides, we are going to remove all punctuation so all mentions and hashtags will become normal words and make all tweets lowercase.

In [ ]:
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))

# After our quick reserach location also doesn't seem to have 
# a strong influence so we will remove the "nolocationplaceholder" token as well.
stop_words.append("nolocationplaceholder")

# Convert to set, so lookup operations are much faster
stop_words = set(stop_words)

In [ ]:
def preprocess(text):

    text = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', ' numbr ',
                  text)

    text = re.sub(r'\d+(\.\d+)?', ' numbr ',
                  text)

    text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', ' httpaddr ',
                  text)

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", text)

    # convert to lower case and split
    words = word_tokenize(letters_only_text.lower())

    # remove stopwords
    cleaned_words = []
    for word in words:
        if word not in stop_words:
            cleaned_words.append(word)

    sen = ' '.join(cleaned_words)

    return sen

In [ ]:
df["cleaned_text"] =df.text_loc.swifter.apply(preprocess)

In [ ]:
df

### Rare word removal

If a word occurs once or twice in the whole dataset, its occurrence doesn't change much. However, since these words are still "analyzed" by the classifier, they may in fact violate the classification process! It is common to remove such words. Let's start with creating a dictionary of the word counts - basically a term frequency dictionary.

In [ ]:
app_desc_clean_list = df["cleaned_text"].tolist()
whole_corpus = " ".join(app_desc_clean_list)

In [ ]:
from collections import Counter
rare_words = {} # again, storing it in dictionary instead of list will make looking through it much faster

counter_dic = Counter(whole_corpus.split())

Let's define a word as "rare" if it occurs only once or twice in the corpus.

In [ ]:
for (key,value) in counter_dic.items():
    if value < 3:
        rare_words[key] = value

Let's explore which words are actually "rare".

In [ ]:
rare_words

In [ ]:
len(rare_words)

In [ ]:
len(set(corpus))

Now, we can delete them.

In [ ]:
def delete_rare_words(text):
    words = word_tokenize(text)

    cleaned_words = []
    
    for word in words:
        if word not in rare_words:
            cleaned_words.append(word)
            
    return ' '.join(cleaned_words)

In [ ]:
df["cleaned_text"] = df["cleaned_text"].swifter.apply(delete_rare_words)

In [ ]:
df

One more thing - if there are any duplicates let's delete them as well.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df

On the other hand, we may also want to see the most frequent words. Let's use the `wordcloud` to visualize it!

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
 
# Create a list of word
text=whole_corpus
 
# Create the wordcloud object
wordcloud = WordCloud(width=480, height=480, background_color='white').generate(text)
 
# Display the generated image:
plt.imshow(wordcloud,  interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

### Textual labels conversion to numeric values

And the very last to do before training classifiers is to encode text labels into numeric labels. Some classifiers may not accept textual labels.

In [ ]:
possible_labels = df.airline_sentiment.unique()
possible_labels

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

# We can refer to this mapping later
label_dict

In [ ]:
df['label'] = df.airline_sentiment.replace(label_dict)

In [ ]:
df

---

## Latent Semantic Analysis (LSA) visualization

Latent Semantic Analysis is a tool for grouping documents with similar meanings. We can use it to visualize the current dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.cleaned_text, df.label, test_size=0.2, random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 1), binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib
import matplotlib.patches as mpatches

def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['orange','green', "red"]
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            orange_patch = mpatches.Patch(color='orange', label='Neutral')
            blue_patch = mpatches.Patch(color='green', label='Positive')
            green_patch = mpatches.Patch(color='red', label='Negative')
            plt.legend(handles=[orange_patch, blue_patch, green_patch], prop={'size': 20})


fig = plt.figure(figsize=(8, 8))          
plot_LSA(X_train_counts, y_train)
plt.show()

Now we are ready for the final confrontation!

---

# Model training

In this section, we are going to train and evaluate different classifiers. Keep in mind that the dataset is very messy and we shouldn't expect astonishing performance improvement.

## Comparing with baseline

Firstly, let's use the same classifier and its configuration, so we can see how our processing and feature engineering changed the accuracy.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.cleaned_text, df.airline_sentiment, test_size=0.2, random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 1), binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
mnb = MultinomialNB()
y_pred = mnb.fit(X_train_counts, y_train)
y_predicted_counts = mnb.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(mnb, X_train_counts, y_train, cv=5)

In [ ]:
scores.mean()

In [ ]:
#old score: 0.7596467191733065

Ok... using the same classifier (and its parameters) as before, we have a slight improvement of 0.5%. This is definitely not sufficient and we want more! 


### Using different vectorizer

Let's consider a different vectorizer.

In [ ]:
count_vectorizer = TfidfVectorizer(ngram_range=(1,1))


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

mnb = MultinomialNB()
y_pred = mnb.fit(X_train_counts, y_train)
y_predicted_counts = mnb.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(mnb, X_train_counts, y_train, cv=5)
scores.mean()

It seems like the TD-IDF doesn't work well in this case. We will stick to the countvectorizer because it gives us a bit better results. 

### Modifying vectorizer parameters

Let's play around with the parameters of the vectorizer before we tweak the classifier and increase the n-gram range. N-grams are joints of n-consecutive words. The range specifies the minimum and the maximum number of words.

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1,3) ,binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
y_pred = mnb.fit(X_train_counts, y_train)
y_predicted_counts = mnb.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(mnb, X_train_counts, y_train, cv=5)
scores.mean()

Increasing the n-gram range also doesn't increase the performance of the classifier. 

---

## Using Logistic Regression

Let's try out another classifier - Logistic Regression.

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1,2) ,binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
clf = LogisticRegression(solver="newton-cg")
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

In [ ]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean()

Nice - logistic regression performs significantly better. 

### Searching for best hyper-parameters

As you know, there are hyper-parameters like the `C` parameter, `solver` or `penalty`. We can try to find the best parameters manually, but it is much better to perform the Grid Search. Let's run it! **Note: This may take a lot of time!**

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'C': np.linspace(0.1, 1, 5), "solver":["newton-cg"]}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(X_train_counts, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

In [ ]:
clf = LogisticRegression(C=0.55, solver='newton-cg' )
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean()

---

## SVM

We will use several configurations for SVM.

In [ ]:
from sklearn import svm
clf = svm.SVC()

clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean()

In [ ]:
clf = svm.SVC(kernel="poly", degree=2)

clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

In [ ]:
clf = svm.SVC(kernel="linear")

clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

Again, we can try to find best parameters for SVM using the Grid Search.

In [ ]:
parameters = {'C': np.linspace(0.1, 5, 5), "kernel":["rbf"]} 
grid_search = GridSearchCV(svm.SVC(), parameters)
grid_search.fit(X_train_counts, y_train)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

---

## Train-test split modification
Another approach is to modify the train-test split. 

### Size changes
Let's make the training set larger and add the stratify parameter.

In [ ]:
# increasing training set size

X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.airline_sentiment, test_size=0.1,
                                                    random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 2), binary=True)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

clf = LogisticRegression(C=0.325, solver='newton-cg')
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" %
      (accuracy, precision, recall, f1))


scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean()  # no big difference - small overfitting on training set

### Resampling
We can also try to randomly oversample positive and neutral tweets to create a more balanced training dataset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.airline_sentiment, test_size=0.2,
                                                    random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 2), binary=True)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='auto')

# Only resample the training dataset
X_train_over, y_train_over = oversample.fit_resample(X_train.to_numpy().reshape(-1,1), y_train)
X_train_over = X_train_over.T[0]

Before resampling...

In [ ]:
sns.catplot(x="airline_sentiment", kind="count", data=pd.DataFrame({"text" : pd.Series(X_train), "airline_sentiment" : pd.Series(y_train)}))

And after...

In [ ]:
sns.catplot(x="airline_sentiment", kind="count", data=pd.DataFrame({"text" : pd.Series(X_train_over), "airline_sentiment" : pd.Series(y_train_over)}))

In [ ]:
X_train_counts = count_vectorizer.fit_transform(X_train_over)
X_test_counts = count_vectorizer.transform(X_test)

clf = LogisticRegression(C=0.325, solver='newton-cg')
clf.fit(X_train_counts, y_train_over)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" %
      (accuracy, precision, recall, f1))


scores = cross_val_score(clf, X_train_counts, y_train_over, cv=5, scoring="f1_weighted")
scores.mean()  # high overfitting on training set

The test set result is didn't change much but as you can see, there is **very high overfitting** when running CV on the training set.

---

# Conclusion

Let's say this is our final model.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.airline_sentiment, test_size=0.2,
                                                    random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 2), binary=True)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

clf = LogisticRegression(C=0.325, solver='newton-cg')
clf.fit(X_train_counts, y_train)

scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean() 

It seems like with linear classifiers we will not get much more than 78% accuracy out of it. 

### Confusion matrix
Of course one should also look at the misclassification. We will use the 3x3 confusion matrix to get some insights.

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf, X_test_counts, y_test) 
plt.show()

### Words with strong influence

Let's inspect which particular words have the highest influence on the classification decision for each class.

In [ ]:
weights_dict = {}
weights_dict["feature"] = count_vectorizer.get_feature_names()
for class_num in range(3):
    weights_dict[clf.classes_[class_num]] = clf.coef_[class_num]

features_weights = pd.DataFrame(weights_dict)

In [ ]:
features_weights.sort_values(by="negative", ascending=False).head(10)

In [ ]:
features_weights.sort_values(by="neutral", ascending=False).head(10)

In [ ]:
features_weights.sort_values(by="positive", ascending=False).head(10)

---

# Further ideas

However, if you are brave enough to continue working with this dataset...

### What happens if we delete the airline info from the tweets so that they don't influence them?

In [ ]:
stop_words = list(stop_words)
# What is interesting, even that tweets are not about "jetblue" airlines, 
# it still occurs quite often in the dataset so let's remove it as well.
stop_words.extend(["virginamerica", "united", "delta", "usairways", "americanair", "southwestair", "jetblue"])  
stop_words = set(stop_words)

In [ ]:
df_no_airlines = df.copy(deep=True)
df_no_airlines["cleaned_text"] =df_no_airlines.text_loc.swifter.apply(preprocess)

Let's visualize what are the most frequent words in the dataset after this change.

In [ ]:
app_desc_clean_list = df_no_airlines["cleaned_text"].tolist()
whole_corpus = " ".join(app_desc_clean_list)

# Create a list of word
text=whole_corpus
 
# Create the wordcloud object
wordcloud = WordCloud(width=480, height=480, background_color='white').generate(text)
 
# Display the generated image:
plt.imshow(wordcloud,  interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_no_airlines.cleaned_text, df_no_airlines.airline_sentiment, test_size=0.2,
                                                    random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1, 2), binary=True)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
clf = LogisticRegression(C=0.325, solver='newton-cg' )
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))


scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean()

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf, X_test_counts, y_test) 
plt.show()

So now the classifier makes fewer mislabels as neutral if something was negative but mislabels neutrals more often as negative. Although it has better accuracy on negatives, on positives the accuracy is lower. However, as we can see, the airline has no significant impact on the sentiment. 

---

### What happens if we change the problem into the binary classification (negative vs non-negative)?

For the previous problem we have created a separate dataframe, so here we are going back to using airline tokens.

In [ ]:
# Previous labels: 0 - neutral, 1 - positive, 2 - negative
# New labels: 1 - non-negative, 2 negative
df["binary_label"] = df.label.apply(lambda x: 1 if x == 0 else x)
df["binary_airline_sentiment"] = df.airline_sentiment.apply(lambda x: "positive" if x == "neutral" else x)

In [ ]:
df

In [ ]:
sns.catplot(x="binary_airline_sentiment", kind="count", data=df)

Now the dataset is much less imbalanced. Although it could be downsampled or oversampled, we will not change it for now.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.binary_airline_sentiment, test_size=0.1, 
                                                                                random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1,1),binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

clf = LogisticRegression(C=0.325, solver='newton-cg')
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))


scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean() 

82%! As you can see, even on the first attempt we can get a much higher score. However, we had to pay for it by removing a class and transforming the task into the binary decision. We can also try to visualize it using the t-SNE LSA graph.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.binary_label, test_size=0.2, 
                                                                                random_state=40)
count_vectorizer = CountVectorizer(ngram_range=(1,1),binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib
import matplotlib.patches as mpatches

def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        color_mapper = {label:idx for idx,label in enumerate(set(test_labels))}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['green', "red"]
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=test_labels, cmap=matplotlib.colors.ListedColormap(colors))
            blue_patch = mpatches.Patch(color='green', label='Positive')
            green_patch = mpatches.Patch(color='red', label='Negative')
            plt.legend(handles=[blue_patch, green_patch], prop={'size': 20})


fig = plt.figure(figsize=(8, 8))          
plot_LSA(X_train_counts, y_train)
plt.show()

---

### What happens if we use descriptions of emojis as additional features?

The `emoji` python package contains descriptions of each emoji. Since these descriptions often express emotions ("**sad** face", "**happy** face"), we can try to use them instead of marking tweets containing negative emojis. Of course, in our case emojis are present in an extremely small number of tweets, so the improvement won't be significant. However, in other applications, it may result in better performance. Let's see how it works with the following example.

In [ ]:
def emojis_meaning(text):
    result = ""
    emojis =  ''.join(c for c in text if c in emoji.UNICODE_EMOJI['en'])
    for emo in emojis:
        result += re.sub(r'_', " ", emoji.demojize(emo)[1:-1]) + " "
    return result

In [ ]:
emojis_meaning("😀👍")

In [ ]:
emojis_meaning("@VirginAmerica 👍 Need to start flying to @KCIAirport .  😊😀😃😄")

In [ ]:
df['emojis_meaning'] = df.text_loc.apply(emojis_meaning)

Since we need to use raw data again, I will paste data processing in the cell below. Nothing new, we've already done it above - this is all the processing we did, compressed in one cell.

In [ ]:
df = df_tweets[[ "airline_sentiment", "airline", "text", "tweet_location"]]
df["location_ner"] = df.tweet_location.swifter.apply(filter_location)
location_df = df["location_ner"].apply(pd.Series)
df = pd.concat([location_df, df], axis=1)
df.rename(columns={0: 'loc_1', 1: 'loc_2', 2: "loc_3", 3: "loc_4"}, inplace="True")
df["text_loc"] = df["text"].astype(str) + " " + df["loc_1"].astype(str)
df = df[["text_loc", "airline_sentiment" ]]
df["neg_emoji"] = df.text_loc.apply(emojis_meaning)  # change here to "contains_neg_emoji" if you want to use negative emojis again
df["text_loc"] = df["text_loc"].astype(str) + " " + df["neg_emoji"].astype(str)
df = df[["text_loc", "airline_sentiment"]]
df["cleaned_text"] =df.text_loc.swifter.apply(preprocess)
app_desc_clean_list = df["cleaned_text"].tolist()
whole_corpus = " ".join(app_desc_clean_list)
rare_words = {}
counter_dic = Counter(whole_corpus.split())
for (key,value) in counter_dic.items():
    if value < 3:
        rare_words[key] = value
df["cleaned_text"] = df["cleaned_text"].swifter.apply(delete_rare_words)
df = df.drop_duplicates()

Let's train the classifier!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.cleaned_text, df.airline_sentiment, test_size=0.1, 
                                                                                random_state=40, stratify=df.airline_sentiment)
count_vectorizer = CountVectorizer(ngram_range=(1,2),binary=True)


X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

clf = LogisticRegression(C=0.325, solver='newton-cg' )
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)
accuracy, precision, recall, f1 = get_metrics(y_test, y_predicted_counts)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))


scores = cross_val_score(clf, X_train_counts, y_train, cv=5)
scores.mean() 

As you can see, it works a bit better, but since emojis are only in about 1% of all tweets, this modification doesn't have a strong effect on the classification.

## This is the end of Notebook 6

Feel free to experiment with this dataset more! If you are interested in the most efficient algorithms developed to deal with this problem, the airline classification dataset has been used by many researchers and you can find many papers about it online. 